# Liam Cottrell

# First time

If this is the first time you have run this program, please run the following code block. We plan to make use of the ggplot package in python, which will mean that we need to install it into our Annoconda Enviroment. Due to changes in the way that pandas methods have been updated, we will also need to update aspects of ggplot to be functional.

#### Uncomment the bellow block of code first and run, after which, please re comment the block.

After you perform these actions, perform a "Restart and Run all" option in the Kernal menu in your Jupyter Notebook.

In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} ggplot
# !sed -i "s,from pandas.lib import Timestamp,from pandas import Timestamp,g" /opt/conda/lib/python3.6/site-packages/ggplot/stats/smoothers.py
# !sed -i "s,smoothed_data.sort('x'),smoothed_data.sort_values('x'),g" /opt/conda/lib/python3.6/site-packages/ggplot/stats/stat_smooth.py

### Imports for Application

In [ ]:
# import all our libraries here
%matplotlib inline 
import matplotlib.pyplot as plt # this library will be used for the scatter plot
import numpy as np 
import pandas as pd # to manage data frames and reading csv files
import random
import pprint

from ggplot import *

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
# np.warnings.filterwarnings('ignore')

In [ ]:
seed = 256712
np.random.seed(seed)

In [ ]:
def create_data_wrapper(dataset, training):
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    my_inputs = [np.reshape(x, (len(dataset.iloc[:, 1:].columns), 1)) for x in dataset.values[:, 1:]]
    if training:
        my_results = [vectorized_result(y) for y in dataset.iloc[:,0]]
    else:
        my_results = dataset.iloc[:,0]
    compliled_data = list(zip(my_inputs, my_results))
    return compliled_data

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

### ANN Network Class

Class adapted from Repo found here: https://github.com/mnielsen/neural-networks-and-deep-learning

In [ ]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, test_data, epochs, mini_batch_size, eta, verbose):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        self.training_data = training_data
        n = len(self.training_data)
        
        return_data = []

        if test_data:
            self.my_test_data = test_data
            n_test = len(self.my_test_data)
        else:
            self.test_data = None

        for j in range(epochs):
            prepare_data = {}
            prepare_data['epoch'] = j
            prepare_data['data_quantity'] = n_test
#             random.shuffle(self.training_data)
            mini_batches = [
               self.training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                prepare_data['correct_quantity'] = self.evaluate(self.my_test_data)
#                 print(prepare_data['correct_quantity'])
                prepare_data['accuracy'] = (prepare_data['correct_quantity']/prepare_data['data_quantity'])*100
                if verbose:
                    print("Epoch {} : {} / {}".format(j,self.evaluate(self.my_test_data),n_test));
            else:
                prepare_data['correct_values'] = None
                if verbose:
                    print("Epoch {} complete".format(j))
            return_data.append(prepare_data)
                
        return return_data

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, comparison_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
#         print(comparison_data)
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in comparison_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
wine_df_orig = pd.read_csv('data/wineQualityWhites.csv')
wine_df_orig.head()

In [ ]:
# remove first coloumn as just a repeat of the index.
wine_df_filtered = wine_df_orig.iloc[:, 1:]
# wine_df_filtered.head()

In [ ]:
wine_df_filtered.hist(bins=50, figsize=(20,15))
plt.show

In [ ]:
# Create a 80
# wine_train_valid_raw, wine_test_raw = np.split(wine_df_filtered, [int(.80*len(wine_df_filtered.index))])
# wine_train_raw, wine_valid_raw = np.split(wine_train_valid_raw, [int(.80*len(wine_train_valid_raw.index))])

wine_train_raw, wine_test_raw = np.split(wine_df_filtered, [int(.80*len(wine_df_filtered.index))])
# wine_train = len(wine_df.index)
print(len(wine_train_raw.index))
# print(len(wine_valid_raw.index))
print(len(wine_test_raw.index))

In [ ]:
wine_train_raw.head()

In [ ]:
wine_factors_train = wine_train_raw.iloc[:, 0:len(wine_train_raw.columns)]
wine_quality_train = wine_train_raw.iloc[:, (len(wine_train_raw.columns)-1)]
wine_factors_test = wine_test_raw.iloc[:, 0:len(wine_test_raw.columns)]
wine_quality_test = wine_test_raw.iloc[:, (len(wine_test_raw.columns)-1)]

In [ ]:
sc = StandardScaler()
wine_factors_train[wine_factors_train.columns] = sc.fit_transform(wine_factors_train[wine_factors_train.columns])
wine_factors_test[wine_factors_test.columns] = sc.transform(wine_factors_test[wine_factors_train.columns])

In [ ]:
wine_train_formatted = wine_factors_train
wine_train_formatted[wine_quality_train.name] = wine_quality_train

wine_test_formatted = wine_factors_test
wine_test_formatted[wine_quality_test.name] = wine_quality_test

In [ ]:
# rearange the last coloumn to become the first
wine_cols = wine_train_formatted.columns.tolist()
wine_cols = wine_cols[-1:] + wine_cols[:-1]
wine_train_formatted = wine_train_formatted[wine_cols]
wine_test_formatted = wine_test_formatted[wine_cols]

In [ ]:
wine_test_formatted.head()

In [ ]:
wine_train= create_data_wrapper(wine_train_formatted, True)
wine_test = create_data_wrapper(wine_test_formatted, False)

Small epoch value to see effect on model accuaracy.

In [ ]:
small_epoch_net = Network([11, 50, 10])
small_epoch_on_accuracy = small_epoch_net.SGD(wine_train, wine_test, 100, 10, 0.3, False)

In [ ]:
small_epoch_on_accuracy_df = pd.DataFrame(small_epoch_on_accuracy)
small_epoch_on_accuracy_df

In [ ]:
ggplot(aes(x='epoch', y='accuracy'), data=small_epoch_on_accuracy_df) +\
    geom_line() +\
    stat_smooth(color='blue', span=0.2)

Going to Test Model Accuracy depedning on the quantity of Epoch itterations.

Data will start at 0, then progress to 1000 epochs.

In [ ]:
large_epoch_net = Network([11, 5, 10])
large_epoch_on_accuracy = large_epoch_net.SGD(wine_train, wine_test, 1000, 10, 0.3, False)

In [ ]:
# pprint.pprint(epoch_on_accuracy)
large_epoch_on_accuracy_df = pd.DataFrame(large_epoch_on_accuracy)
large_epoch_on_accuracy_df

In [ ]:
ggplot(aes(x='epoch', y='accuracy'), data=large_epoch_on_accuracy_df) +\
    geom_line() +\
    stat_smooth(color='blue', span=0.8)